In [3]:
Stint.objects.count()

0

In [ ]:
import re
import pprint

def find_staff(text):
    positions = ('daglig leder', 'redaktør', 'desksjef', 'nyhetsleder', 'nettredaktør',
                 'fotosjef', 'debattredaktør', 'magasinredaktør')
    pospat = f'({"|".join(positions)})' 
    pattern = re.compile(
        r'^(?P<title>' + pospat + r'):\s*(?P<name>[-\w ]+)$', flags=re.M | re.I)
    matches = pattern.finditer(text)
    output = {
        d['title']: d['name']
        for d in [m.groupdict() for m in matches]
        if d['title'] in positions
    }
    print(output)
    return output

In [68]:
def find_staff(text):
    positions = ['daglig leder', '\w*redaktør', '\w*leder', '\w*sjef' ]
    pattern = r'^(?P<title>' f'({"|".join(positions)})' r'):\s*(?P<name>.+)$'
    regex = re.compile(pattern, flags=re.M | re.I)
    matches = regex.finditer(text.replace('\x08', ''))
    return { d['title'].lower(): d['name'] for d in [m.groupdict() for m in matches] }

content = content.replace('\x08', '')
print(re.findall(r'^\w+sjef:.*', content, flags=re.M | re.I))
find_staff(content)


['fotosjef:  Ketil Blom', 'desksjef:  Håken Lid']


{'anmelderredaktør': 'Marie De Rosa',
 'daglig leder': 'Katrine Myra',
 'debattredaktør': 'Dag Grytli',
 'desksjef': 'Håken Lid',
 'featureredaktør': 'Jørgen Brynhildsvoll',
 'fotosjef': 'Ketil Blom',
 'kulturredaktør': 'Maren Ørstavik',
 'nettredaktør': 'Mikael Lunde',
 'nyhetsredaktør': 'Mathias Vedeler',
 'redaksjonsleder': 'Nordis Tennes',
 'redaktør': 'Simen Tallaksen'}

In [71]:
def get_content(print_issue):
    try:
        return print_issue.get_page_text_content(2,100)
    except Exception as e:
        print(e)
        return ''

def find_positions(print_issue):
    content = get_content(print_issue)
    return find_staff(content)

missing = []

def add_to_stint(title, name, date):
    try:
        contributor = Contributor.objects.get(display_name=name)
    except Contributor.DoesNotExist:
        contributor = Contributor.get_or_create(name)[0]
        print(f'could not find {name}, found {contributor}')
    position, _ = Position.objects.get_or_create(title=title)
    defaults = {'start_date': date, 'end_date': date}
    stint, _ = Stint.objects.get_or_create(
        defaults=defaults, position=position, contributor=contributor)
    stint.end_date = max(stint.end_date, date)
    stint.start_date = min(stint.start_date, date)
    stint.save()
    return stint


def find_stints_from_pdfs(qs=None):
    if qs is None:
        qs = PrintIssue.objects.all()
    for print_issue in qs:
        data = find_positions(print_issue)
        print(print_issue)
        print(', '.join(f'{p}: {data[p]}' for p in data))
        date = print_issue.issue.publication_date
        for title, name in data.items():
            add_to_stint(title, name, date)

find_stints_from_pdfs()

http://static.universitas.no/media/pdf/universitas_2015-30_mag.pdf
magasinredaktør: Thea Storøy Elnan, magasinfotosjef: Iselin Shaw of Tordarroch
http://static.universitas.no/media/pdf/universitas_2011-11.pdf
daglig leder: Katrine Myra, redaktør: Simen Tallaksen, redaksjonsleder: Nordis Tennes, fotosjef: Ketil Blom, desksjef: Håken Lid, nettredaktør: Mikael Lunde, nyhetsredaktør: Mathias Vedeler, debattredaktør: Dag Grytli, kulturredaktør: Maren Ørstavik, featureredaktør: Jørgen Brynhildsvoll, anmelderredaktør: Marie De Rosa
http://static.universitas.no/media/pdf/universitas_2014-30.pdf
daglig leder: Louise Faldalen Prytz, redaktør: Geir Molnes, redaksjonsleder: Vilde Sagstad Imeland, desksjef: Håkon Sukuvara, nettredaktør: Petter Fløttum, magasinredaktør: Thea Storøy Elnan, nyhetsredaktør: Eirik Omvik, kulturredaktør: Eirik Billingsø Elvevold, reportasjeredaktør: Vilde Sagstad Imeland, utenriksredaktør: Ingri G. Bergo, debattredaktør: Anders Sondrup, anmelderredaktør: Magnus Newth
c

http://static.universitas.no/media/pdf/universitas_2016-9.pdf
daglig leder: Louise Faldalen Prytz, redaktør: Magnus Newth, redaksjonsleder: Torgeir Mortensen, fotosjef: Amanda O. Berg, desksjef: Lise Blekastad, nettredaktør: Signe Rosenlund-Hauglid, magasinredaktør: Ingrid Bergo, nyhetsredaktør: Torgeir Mortensen, debattredaktør: Hanna Skotheim, kulturredaktør: Kaja Storrøsten, anmelderredaktør: Pia Sandved Berg
could not find Ingrid Bergo, found Ingri Bergo
http://static.universitas.no/media/pdf/universitas_2016-10.pdf
daglig leder: Louise Faldalen Prytz, redaktør: Erik, redaksjonsleder: Torgeir Mortensen, fotosjef: Amanda O. Berg, desksjef: Lise Blekastad, nettredaktør: Signe Rosenlund-Hauglid, magasinredaktør: Ingri Bergo, nyhetsredaktør: Torgeir Mortensen, utenriksredaktør: Knut Arne Oseid, debattredaktør: Hanna Skotheim, kulturredaktør: Kaja Storrøsten, anmelderredaktør: Pia Sandved Berg
could not find Erik, found Erik
http://static.universitas.no/media/pdf/universitas_2014-03.pdf

http://static.universitas.no/media/pdf/universitas_2016-24_mag.pdf
magasinredaktør: Kaja Storrøsten, magasinfotosjef: Kristina Kvammen
http://static.universitas.no/media/pdf/universitas_2016-24.pdf
daglig leder: Louise Faldalen Prytz, redaktør: Torgeir Mortensen, nyhetsleder: Birk Tjeldflaat Helle, fotosjef: Adrian Nielsen, desksjef: Lise Blekastad, nettredaktør: Sondre Myhre, magasinredaktør: Kaja Storrøsten, utenriksredaktør: Eskil Wie, debattredaktør: Knut Arne Oseid, anmelderredaktør: Morten Oftedal Schwencke
http://static.universitas.no/media/pdf/universitas_2016-25.pdf
daglig leder: Louise Faldalen Prytz, redaktør: Torgeir Mortensen, nyhetsleder: Birk Tjeldflaat Helle, fotosjef: Adrian Nielsen, desksjef: Lise Blekastad, nettredaktør: Sondre Myhre, magasinredaktør: Kaja Storrøsten, kulturredaktør: Philip A. Johannesborg, debattredaktør: Knut Arne Oseid, anmelderredaktør: Morten Oftedal Schwencke
could not find Philip A. Johannesborg, found Philip André Johannesborg
http://static.u

http://static.universitas.no/media/pdf/b99ba836edaa11e68c22f23c916e5154/universitas_2017-5.pdf
daglig leder: Louise Faldalen Prytz, redaktør: Torgeir Mortensen, nyhetsleder: Birk Tjeldflaat Helle, fotosjef: Adrian Nielsen, desksjef: Lise Blekastad, nettredaktør: Sondre Myhre, magasinredaktør: Signe Rosenlund-Hauglid, utenriksredaktør: Eskil Wie, kulturredaktør: Morten Oftedal Schwencke, debattredaktør: Knut Arne Oseid, anmelderredaktør: Regine Stokstad
http://static.universitas.no/media/pdf/e093fe7cf32a11e68c22f23c916e5154/universitas_2017-6.pdf
daglig leder: Louise Faldalen Prytz, redaktør: Torgeir Mortensen, nyhetsleder: Birk Tjeldflaat Helle, fotosjef: Adrian Nielsen, desksjef: Lise Blekastad, nettredaktør: Sondre Myhre, magasinredaktør: Signe Rosenlund-Hauglid, kulturredaktør: Morten Oftedal Schwencke, debattredaktør: Knut Arne Oseid, anmelderredaktør: Regine Stokstad
http://static.universitas.no/media/pdf/b1a145880eab11e7aa6bf23c916e5154/universitas_2017-11_mag.pdf
magasinredaktør

http://static.universitas.no/media/pdf/universitas_2012-25.pdf
daglig leder: Monica Reigstad, redaktør: Magnus Lysberg, redaksjonsleder: Gabriel Steinsbekk, fotosjef: Skjalg Bøhmer Vold, desksjef: Benjamin Edward Oliver, nettredaktør: Heljar Havnes, nyhetsredaktør: Ingvild Sagmoen, debattredaktør: Thorbjørn K. Borlaug, kulturredaktør: Guro Havro Bjørnstad, reportasjeredaktør: Agnes Klem, anmelderredaktør: Peder D. Stabell
could not find Skjalg Bøhmer Vold, found Skjalg Vold
could not find Benjamin Edward Oliver, found Benjamin Oliver
could not find Thorbjørn K. Borlaug, found Thorbjørn Kringlebotn Borlaug
could not find Peder D. Stabell, found Peder Stabell
http://static.universitas.no/media/pdf/universitas_2012-10.pdf
daglig leder: Monica Reigstad, redaktør: Magnus Lysberg, redaksjonsleder: Gabriel Steinsbekk, desksjef: Jørgen Brynhildsvoll, nettredaktør: Hans Skjong, debattredaktør: Gabriel Steinsbekk, kulturredaktør: Øyvind Gallefoss, reportasjeredaktør: Ingeborg Amundsen, anmelderr

...

http://static.universitas.no/media/pdf/universitas_2014-13.pdf
daglig leder: Monica Reigstad, redaktør: Anne Fougner Helseth, redaksjonsleder: Are W. Sandvik, desksjef: Håkon Sukuvara, nettredaktør: Thorbjørn Kringlebotn, magasinredaktør: Solveig N. Langvad, nyhetsredaktør: Geir Molnes, debattredaktør: Thea Storøy Elnan, kulturredaktør: Kenneth Wangen, reportasjeredaktør: Christoffer Gundersen, anmelderredaktør: Vilde Sagstad Imeland
could not find Solveig N. Langvad, found Solveig Nygaard Langvad
http://static.universitas.no/media/pdf/universitas_2011-27.pdf
daglig leder: Katrine Myra, redaktør: Simen Tallaksen, redaksjonsleder: Nordis Tennes, fotosjef: Skjalg Bøhmer Vold, desksjef: Jørgen Brynhildsvoll, nettredaktør: Gabriel Steinsbekk, nyhetsredaktør: Lars Thorvaldsen, debattredaktør: Magnus Lysberg, kulturredaktør: Mirjam Sorge Folkvord, featureredaktør: Anders Fjellberg, anmelderredaktør: Nordis Tennes
could not find Skjalg Bøhmer Vold, found Skjalg Vold
could not find Mirjam Sor

http://static.universitas.no/media/pdf/universitas_2015-03.pdf
daglig leder: Louise Faldalen Prytz, redaktør: Geir Molnes, redaksjonsleder: Vilde Sagstad Imeland, desksjef: Marthe Olstad, nettredaktør: Petter Fløttum, magasinredaktør: Thea Storøy Elnan, nyhetsredaktør: Magnus Newth, debattredaktør: Torgeir G. Mortensen, kulturredaktør: Julie Kalager, reportasjeredaktør: Ingrid Gipling, anmelderredaktør: Benedicte Tobiassen
could not find Torgeir G. Mortensen, found Torgeir Mortensen
could not find Ingrid Gipling, found Ingrid Elise Gipling
http://static.universitas.no/media/pdf/universitas_2010-24.pdf
redaktør: Anders ­, redaksjonsleder: Anna Werenskjold, fotosjef: Ketil Blom, desksjef: Håken Lid, nettredaktør: Øyvind Bosnes Engen, daglig leder: Monica Reigstad, nyhetsredaktør: Simen Tallaksen, debattredaktør: Anna Werenskjold, kulturredaktør: Anders R. Christensen, featureredaktør: Nordis Tennes, anmelderredaktør: Kjetil Nordengen
http://static.universitas.no/media/pdf/universitas_20

http://static.universitas.no/media/pdf/universitas_2014-29.pdf
daglig leder: Louise Faldalen Prytz, redaktør: Geir Molnes, redaksjonsleder: Vilde Sagstad Imeland, desksjef: Håkon Sukuvara, nettredaktør: Petter Fløttum, magasinredaktør: Thea Storøy Elnan, nyhetsredaktør: Eirik Omvik, debattredaktør: Anders Sondrup, kulturredaktør: Eirik Billingsø Elvevold, reportasjeredaktør: Vilde Sagstad Imeland, anmelderredaktør: Magnus Newth, magasinfotosjef: Henrik Evertsson
http://static.universitas.no/media/pdf/universitas_2011-02.pdf
redaktør: Simen Tallaksen, redaksjonsleder: Nordis Tennes, fotosjef: Ketil Blom, desksjef: Håken Lid, nettredaktør: Mikael Lunde, daglig leder: Monica Reigstad, nyhetsredaktør: Mathias Vedeler, debattredaktør: Dag Grytli, kulturredaktør: Maren Ørstavik, featureredaktør: Jørgen Brynhildsvoll, anmelderredaktør: Marie De Rosa
http://static.universitas.no/media/pdf/universitas_2014-16.pdf
daglig leder: Monica Reigstad, redaktør: Anne Fougner Helseth, redaksjonsleder: 

could not find Ingrid Bergo, found Ingri Bergo
http://static.universitas.no/media/pdf/universitas_2015-20.pdf
daglig leder: Louise Faldalen Prytz, redaktør: Magnus Newth, redaksjonsleder: Julie Kalager, fotosjef: Patrick Da Silva Sæther, desksjef: Marthe Olstad, nettredaktør: Magnus Braaten, magasinredaktør: Thea Storøy Elnan, nyhetsredaktør: Torgeir Mortensen, debattredaktør: Anders Veberg, kulturredaktør: Pia Sandved Berg, anmelderredaktør: Benedicte Tobiassen
could not find Patrick Da Silva Sæther, found Patrick da Silva Sæther
http://static.universitas.no/media/pdf/universitas_2015-23.pdf
daglig leder: Louise Faldalen Prytz, redaktør: Magnus Newth, redaksjonsleder: Julie Kalager, fotosjef: Patrick Da Silva Sæther, desksjef: Marthe Olstad, nettredaktør: Magnus Braaten, magasinredaktør: Thea Storøy Elnan, nyhetsredaktør: Torgeir Mortensen, debattredaktør: Anders Veberg, utenriksredaktør: Ingrid Bergo, kulturredaktør: Pia Sandved Berg, anmelderredaktør: Benedicte Tobiassen
could not f

http://static.universitas.no/media/pdf/bbbbe6bc2a8111e78b3c0242ac120008/universitas_2017-12.pdf
daglig leder: Joakim Stene Preston, redaktør: Torgeir Mortensen, nyhetsleder: Birk Tjeldflaat Helle, fotosjef: Adrian Nielsen, desksjef: Lise Blekastad, nettredaktør: Sondre Myhre, magasinredaktør: Signe Rosenlund-Hauglid, utenriksredaktør: Eskil Wie, kulturredaktør: Morten Oftedal Schwencke, debattredaktør: Knut Arne Oseid, anmelderredaktør: Regine Stokstad
http://static.universitas.no/media/pdf/ce5ee046406411e7961e0242ac120008/universitas_2017-15.pdf
daglig leder: Joakim Stene Preston, redaktør: Torgeir Mortensen, nyhetsleder: Birk Tjeldflaat Helle, fotosjef: Adrian Nielsen, desksjef: Lise Blekastad, nettredaktør: Sondre Myhre, magasinredaktør: Signe Rosenlund-Hauglid, utenriksredaktør: Eskil Wie, kulturredaktør: Morten Oftedal Schwencke, debattredaktør: Knut Arne Oseid, anmelderredaktør: Regine Stokstad
http://static.universitas.no/media/pdf/1134cc2a2fb511e78b3c0242ac120008/universitas_20

In [76]:
Byline.objects.filter(story__story_type__section__title='Kultur', credit='by').update(credit='text')
Byline.objects.filter(story__story_type__section__title='Anmeldelser', credit='by').update(credit='text')
Byline.objects.filter(story__story_type__section__title='Nyheter', credit='by').update(credit='text')
Byline.objects.filter(story__story_type__section__title='Debatt', credit='text').update(credit='by')

0

In [77]:
def add_stints_from_bylines(title, credit, exclude=['Debatt']):
    """Add stints based on byline credits"""
    position, _ = Position.objects.get_or_create(title=title)
    for person in Contributor.objects.all():
        bylines = person.byline_set.filter(
            credit__contains=credit
        ).exclude( # unpublished
            story__publication_date=None
        ).exclude( # no opinion pieces
            story__story_type__section__title__in=exclude
        ).order_by(
            'story__publication_date'
        )
        if bylines.count() > 1:  # ignore people with just one byline
            first_byline = bylines.first().story.publication_date.date()
            last_byline = bylines.last().story.publication_date.date()
            stint, created = Stint.objects.get_or_create(
                dict(start_date=first_byline, end_date=last_byline),
                contributor=person, position=position)
            if not created:
                stint.start_date = min(stint.start_date, first_byline)
                stint.end_date = max(stint.end_date, last_byline)
                stint.save()
            print(f'{stint} {bylines.count()}')
       
    
add_stints_from_bylines('journalist', 'text')
add_stints_from_bylines('fotograf', 'photo')
add_stints_from_bylines('oversetter', 'translation')
add_stints_from_bylines('illustratør', 'illustration')

journalist Caroline Rugeldal 63
journalist Hanna Skotheim 44
journalist Maria Reinertsen 64
journalist Henrik Evertsson 7
journalist Benedicte Tobiassen 63
journalist Kjetil Nordengen 45
journalist Ketil Blom 12
journalist Linda Johnsen Ørstavik 48
journalist Lise Grønskar 46
journalist Melike Leblebicioğlu 25
journalist Mathias Vedeler 52
journalist Kirsty Johansen 9
journalist Hallvard Østtveit Barbogen 20
journalist Hans Dalane-Hval 14
journalist Birte Nystad Magnussen 2
journalist Cecilia Johansen Eriksen 4
journalist Magnus Lysberg 25
journalist Julie Nordby Egeland 28
journalist Ørjan Ryland 5
journalist Eirik Omvik 87
journalist Anders Ballangrud 74
journalist Are W. Sandvik 66
journalist Emma Tollersrud 14
journalist Maria Terese Kittilsen 37
journalist Journalist I Universitas 9
journalist Jonas Øren 7
journalist Anders R. Christensen 245
journalist Camilla Brønnich Eikeland 6
journalist Mina Gerhardsen 2
journalist Lars Rønn 6
journalist Tika Sofia León 8
journalist Mads Rand

journalist Nordis Tennes 55
journalist Andreas Sveen Bjørnstad 18
journalist Marie De Rosa 70
journalist Øyvind Bosnes Engen 158
journalist Eir Torvik 56
journalist Ingvild Jacobsen 38
journalist Anders Mathias Johansen 14
journalist Amund Trellevik 30
journalist Vegard Tjørhom 15
journalist Translation By Ingrid F. Brubaker 6
journalist Marianne Aasen 2
journalist Universitas’ Matpanel 3
journalist Tone Magni F. Vestheim 2
journalist Karoline Sanna 8
journalist Dag Grytli 34
journalist Lise Marie Lunaas Holt 16
journalist Nareen Mizoory 7
journalist Caroline Furru 4
journalist Cecilie Einarson Pérez 9
journalist Martin Grevstad 14
journalist Mari K. S. Gjelstad 12
journalist Marit M. Simonsen 6
journalist Skjalg Vold 13
journalist Universitas’ Sommerøltestpanel 2
journalist Bendik Løve 52
journalist Mikael Lunde 31
journalist Anne Høyer 6
journalist Heidi Bade 19
journalist Stella Heieren 7
journalist Aurora Hannisdal 6
journalist Tonje Næss 16
journalist Per Harald Borgen 6
journalis

fotograf Lovise Steinkjellå 2
fotograf Eirik Bryhn Jacobsen 7
fotograf Sara Nordby 4
fotograf Bjørn Brekke 21
fotograf Berit Andreassen 10
fotograf Khai Nguyen 4
fotograf Åshild Støylen 195
fotograf Håkon Mosvold 83
fotograf Håkon Rasmus Bergseth 6
fotograf Solveig Schanke Eikum 9
fotograf Halvor Finess Tretvoll 4
fotograf Fredrik Arff 50
fotograf Håkon Mosvold Larsen 10
fotograf Erik Møller Solheim 2
fotograf Mona Tandsether 12
fotograf Sunniva Trengereid 8
fotograf Audun Kolstad Wiig 2
fotograf Ellinor Bente Dalbye 109
fotograf Øyvind Lie 2
fotograf Elisabeth Johansen 16
fotograf Svein Egil Hatlevik 2
fotograf Espen Gleditsch 2
fotograf Lise Falck 24
fotograf Ida Marie Jahr 2
fotograf Iván Kverme 48
fotograf Ingjerd Sørlie 40
fotograf Elisabeth Sperre Alnes 4
fotograf Lilly Susanne Stensland 31
fotograf Aslak Johannessen 189
fotograf Camilla Tully 13
fotograf Lene Hoem Røysum 19
fotograf Hanne Hvattum 65
fotograf Leif Martin Kirknes 11
fotograf Kristine Gramstad 24
fotograf Julie Hoe

In [79]:
from django.utils import timezone

def update_active(inactive_limit=timezone.timedelta(days=150)):
    today = timezone.now().date()
    for contributor in Contributor.objects.all():
        all_bylines = contributor.byline_set.exclude(story__publication_date=None)
        debatt_bylines = all_bylines.filter(story__story_type__section__title='Debatt')
        status = Contributor.UNKNOWN
        if all_bylines.count() == 0:
            print(contributor)
        elif all_bylines.count() == debatt_bylines.count():
            status = contributor.EXTERNAL
        elif contributor.stint_set.count():
            latest = contributor.stint_set.latest('end_date')
            if today - latest.end_date > inactive_limit:
                status = contributor.RETIRED
            else:
                status = contributor.ACTIVE
        if status != contributor.status:
            contributor.status=status 
            contributor.save(update_fields=['status'])
        
        print(f'{str(contributor):<40} {contributor.get_status_display():<10} {contributor.byline_set.count():>5}')

update_active()
unknowns = Contributor.objects.filter(status=Contributor.UNKNOWN)
active = Contributor.objects.filter(status=Contributor.ACTIVE)
retired = Contributor.objects.filter(status=Contributor.RETIRED)
print(unknowns.count(), active.count(), retired.count())

Nestleder I Studentparlamentet Ved Uio
Nestleder I Studentparlamentet Ved Uio   Ukjent         0
Sundseth
Sundseth                                 Ukjent         0
Brenna
Brenna                                   Ukjent         0
Emil Øversveen                           Ukjent         1
Fotografens Navn
Fotografens Navn                         Ukjent         0
Grønn Liste
Grønn Liste                              Ukjent         0
Se–Man                                   Ukjent         2
Lady Stephanie Huerta Hernandez          Ukjent         1
& Video: Erlend Dalhaug Daae
& Video: Erlend Dalhaug Daae             Ukjent         0
Sunniva Skjeggestad Foto Odin Drønen
Sunniva Skjeggestad Foto Odin Drønen     Ukjent         0
Nyhetsleder Universitas
Nyhetsleder Universitas                  Ukjent         0
Stine Marie Korsfur                      Ukjent         1
Hanna Skotheim                           Aktiv         47
Caroline Rugeldal                        Slutta        63
Helge Leander 

Lars Rønn                                Slutta         7
Ingeborg Misje Illustrasjon Tanjya Silvestrini
Ingeborg Misje Illustrasjon Tanjya Silvestrini Ukjent         0
Moabit Hilft                             Ukjent         1
Morten Oftedal Schwenke
Morten Oftedal Schwenke                  Ukjent         0
Morten Schwencke Foto Raisa Posanger
Morten Schwencke Foto Raisa Posanger     Ukjent         0
Professor Andersen
Professor Andersen                       Ukjent         0
Edvard Brudeli Moen
Edvard Brudeli Moen                      Ukjent         0
Petter Holstad                           Ukjent         1
Olav Røise                               Ekstern        1
N. N.
N. N.                                    Ukjent         0
Teks T
Teks T                                   Ukjent         0
 Prof. Ingrid Os                         Ekstern        1
Eirik Braathen                           Ekstern        1
Dziugas Baltrunas                        Ukjent         1
Camilla Pedelino Ek
Ca

Sverre Stausland Johnsen                 Ekstern        1
Hedvig Haugerud                          Ekstern        1
Stine G. Bergo                           Aktiv         13
Inger Sandlie                            Ekstern        2
Simen August Askeland
Simen August Askeland                    Ukjent         0
Rachel Elisabeth Antonsen                Aktiv          6
Sara Nordby                              Slutta         4
Maria Kjos Fonn                          Aktiv          3
Patrick Egersborg                        Ekstern        1
Camilla Wilhelmsen                       Ekstern        1
Jonas Virtanen
Jonas Virtanen                           Ukjent         1
Guy Thomas Kværnberg                     Ekstern        1
Terje Olav Moen                          Ekstern        1
Anne Helene Bakke                        Ekstern        1
Karl Henrik Storhaug Reinås
Karl Henrik Storhaug Reinås              Ukjent         1
Eirik Jacobsen                           Ukjent         1
Martine

Emil Bull                                Ukjent         1
Burleben                                 Ukjent         3
Salome                                   Ukjent         3
Jannicke Nilsen                          Ukjent         1
Mooziemoo                                Ukjent         1
Ulurk                                    Ukjent         1
Illustrasjon Bjarne Henning Kvåle        Ukjent         1
Nr 16                                    Ukjent         1
Master Of Oslo                           Ukjent         1
Tiden                                    Ukjent         1
Arkiv                                    Ukjent         1
Sunniva Trengereid                       Slutta         8
Claes–Borger Landfald                    Ekstern        1
Anne–Mette Vibe                          Ekstern        1
Arild Pedersen                           Ekstern        6
Undervisningsleder                       Ekstern        2
Ola Elvestuen                            Ekstern        1
Alf Helge Grea

Ansjos                                   Ukjent         1
Dr. Vornoff                              Ukjent         3
5. Etasje                                Ukjent         1
Trollkarlen                              Ukjent         1
Kim Sølve Madsen                         Ukjent         1
Aj                                       Ukjent         1
Camilla Tully                            Slutta        14
Michael Brøndbo                          Slutta       210
Ingrid Sommer                            Ekstern        1
Ragnhild C. Haugen                       Slutta         8
Jørgen Svarstad                          Slutta        68
Balkaran Singh                           Slutta        15
Studentparlamentsrepresentant For Sosialdemokraten Ekstern        1
Finn Olav Haga                           Ekstern        1
Zoya Mehmood Shah                        Ekstern        1
Marte Jørgensen                          Ekstern        1
Marielle Minutella                       Ekstern        1
Håva

...

Klara Furuberg                           Ekstern        2
Jens Folland                             Ekstern        2
Øyvind Bugge Solheim                     Ekstern        1
Pål Thygesen                             Ekstern        1
Ida Madsen Hestman                       Slutta        35
Einar Jullum Leiknes                     Ekstern        1
Nik. Brandal                             Ekstern        1
Eirik Vatnøy                             Ekstern        1
Marius Doksheim                          Ekstern        1
Janne M. Log                             Ekstern        3
Nora Zomlot                              Ekstern        1
Rolf V. Olsen                            Ekstern        1
Lars Heltne                              Slutta        23
Bendik Baksaas                           Slutta        39
Mia Caroline Bratz                       Slutta        25
Ingrid Jørgensen                         Slutta         5
Ingvild Sagmoen                          Slutta        47
Liv-Kristin Rø

Helge Schwitters                         Ekstern        7
Borghild Brede                           Slutta         2
Catrine Harila                           Ekstern        2
Camilla Holm                             Ekstern        2
Henning Warloe                           Ekstern        2
Erik Løvold                              Slutta         2
Eli Gunhild By                           Ekstern        2
Petter Aaslestad                         Ekstern        2
Therese Eia Lerøen                       Ekstern       10
Himanshu Gulati                          Ekstern        3
Edle Ravndal                             Ekstern        2
Kristoffer Kaayne Kaalsaas               Slutta        22
Kjersti Ruud Salomonsen                  Slutta        12
Eirin Nordal                             Ekstern        2
Henriette N. Thommessen                  Ekstern        2
Siam Florelius                           Ekstern        2
Jørgen Kvalsvik                          Slutta        22
Thea Storøy El

Palestinakomiteen Blindern               Ekstern        1
Pontus Hanneteg                          Ekstern        3
Torbjørn Røe Isaksen                     Ekstern        3
Vegard R. Lauvdahl                       Ekstern        1
Kristian Bjørke                          Ekstern        1
Jørgen Eriksson Midtbø                   Ekstern        1
Lars Engebretsen                         Ekstern        1
Geir Wiknes                              Ekstern        1
Line Merete Karlsøen                     Ekstern        1
Truls Oftedal Ellingsen                  Ekstern        1
Rannveig Van Iterson                     Ekstern        1
Jakub Pawlak                             Ekstern        1
Caroline Hovland Svensen                 Ekstern        1
Ellen Dalen                              Ekstern        1
Kaisa R. Hautala                         Ekstern        1
Kunnskapsminister Torbjørn Røe Isaksen   Ekstern        2
Berit Rognhaug                           Ekstern        1
Terje Mørland 

Ole-Fredrik Lambertsen                   Slutta        16
Axel Hodnefjeld                          Slutta        12
Julie Kalager                            Slutta        35
Eline Anker                              Slutta         2
Rune Keisuke Kosaka                      Ekstern        1
Ingvild Høystad Stavem                   Ekstern        1
Maya Sol Sørgård                         Ekstern        1
Stine Hesstvedt                          Slutta         9
Ole Marius Gaasø                         Ekstern        2
Magnar Grønvik Müller                    Ekstern        2
Trond Andresen                           Ekstern        1
Pia Ottilia Danielsen                    Ekstern        1
Tobias Lund                              Ekstern        1
Alf Grønli Simensen                      Aktiv         19
Morten Schwencke Foto: Alf Simensen
Morten Schwencke Foto: Alf Simensen      Ukjent         0
Jon Eskil Ramsrud Kindberg               Ekstern        1
Tobias Grimstad                     

Anne Grethe Klunderud                    Ekstern        1
Kjell Lars Berge                         Ekstern        2
Marthe Amanda Vannebo                    Slutta        19
Martin Anders Dahl                       Slutta        25
Ingunn Sandberg                          Ekstern        1
Andreas Johannesen Delsett               Ekstern        2
Adrian Wilhelm Kjølø Tollefsen           Ekstern        1
Sebastian Dahl                           Slutta        37
Solveig Figenschou                       Ekstern        7
Førsteamanuensis På Norges Veterinærhøgskole Ekstern        1
Anders Kvernmo Langset                   Ekstern        2
Steinar Solås Suvatne                    Ekstern        1
Marit Aasmo Albrigtsen                   Slutta         8
Axel Geard Nygaard                       Slutta        73
Eirik Billingsø Elvevold                 Slutta        40
Petter Drange Brønstad                   Slutta        13
Evelyn Pecori                            Slutta        36
Philip And

In [81]:
for person in unknowns:
    print(person)
    for stint in person.stint_set.all():
        print(stint)
    print()
    #for byline in person.byline_set.all():
    #    print(byline, byline.story.publication_date, byline.story.section)

Nestleder I Studentparlamentet Ved Uio

Sundseth

Brenna

Emil Øversveen

Fotografens Navn

Grønn Liste

Se–Man

Lady Stephanie Huerta Hernandez

& Video: Erlend Dalhaug Daae

Sunniva Skjeggestad Foto Odin Drønen

Nyhetsleder Universitas

Stine Marie Korsfur

Sondre Myhre Foto Sjur Stølen

Vilde Boberg Svineng

Philip Johannesborg Foto Håkon Benjaminsen

Kristian Voldnes Foto Sjur Stølen

Debattredaktør I Universitas

N. N.

Journalist Malin Kvande

Øyunn Syrstad Høydal

Republikanerne

Demokratene

Ville Stemt Demokratisk.

Trygve Bernhardt Moen Haaland

Caroline Ulv Johansson

Ingeborg Misje Illustrasjon Tanjya Silvestrini

Moabit Hilft

Morten Oftedal Schwenke

Morten Schwencke Foto Raisa Posanger

Professor Andersen

Edvard Brudeli Moen

Petter Holstad

N. N.

Teks T

Dziugas Baltrunas

Camilla Pedelino Ek

Marius H. Johannessen

Anne Merete Aass

Emili Knutson Foto Dorthe Karlsen

Harald Hoff

Ingeborg Sleipnes Sivertsen

Redaktørens Dom

I Usa

Dorthe Karlsen Og Matthis Kleeb

Ha

In [84]:
# Fjern ubrukte artikkeltyper

for st in StoryType.objects.all():
    if st.story_set.count() == 0:
        print(st)
        st.delete()
        
for sec in Section.objects.all():
    story_types = sec.storytype_set.all()
    if story_types.count() == 0:
        sec.delete()
    print(sec)
    for story_type in story_types:
        print('   ', story_type, story_type.story_set.count())

Nyheter
    Nett 887
    News 702
    Sletta 1
    Utenriks 457
    Nyhet 4618
    Fem på plassen 2
Magasin
    Camillas Kollektion 29
    Reisebilag 18
    Velkomstbilag 64
    Magasin 970
    Petit 273
Kultur
    Akademia 22
    Kultur 2124
Debatt
    SMS 192
    Debatt 40
    Leserbrev 2202
    Kronikk 499
Baksiden
    Ad Notam 289
    Vi spør 323
Side 2
    Øyeblikket 125
    Kommentar 578
    Leder 508
Anmeldelser
    Hva skjer 167
    Anmeldelse 3298
    Ukas Advarsel 1
Feature
    Reportasje 21
    Studietid 956
    Min studietid 345
Annet
    Tassen 9
    Quiz 6
    Universitas Melder 23
